<a href="https://colab.research.google.com/github/humayraR/fact-ai_Repl/blob/main/DPGAN_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# from google.colab import drive
# drive.mount('/content/drive')

In [14]:
# save pipe.pkl to output data folder
# !cp <output-file-name> /content/drive/MyDrive/GitCode

In [15]:
# !pip install table_evaluator
# !pip install ctgan
# !pip install synthcity
# !pip3 install torch torchvision torchaudio

In [16]:
# ! git clone https://github.com/humayraR/fact-ai_Repl.git

In [17]:
# import sys
# sys.path.insert(0,'/content/fact-ai_Repl')

In [18]:
# %cd fact-ai_Repl

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score, recall_score, roc_auc_score

from table_evaluator import TableEvaluator

from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

import os.path, sys
from tests.utils import load_adult

import warnings

# synthcity absolute
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import GenericDataLoader

In [20]:
log.add(sink=sys.stderr, level="INFO")
sys.path.append(os.getcwd())

In [21]:
import PreProcessData

clean_df = PreProcessData.clean_df('data/adult.data')
#arr_X, arr_y, adult_df = load_adult() # arr_X np array without label/target, arr_y np array of just label/target

# clean_df.head()

In [24]:
loader = GenericDataLoader(
    clean_df,
    target_column="label",
    sensitive_columns=["race","sex","native-country"],
)

#loader.dataframe().head()

In [25]:
# train dpgan for e=0.1
from synthcity.plugins.privacy import plugin_dpgan
from Metrics import get_metrics
from synthcity.benchmark import Benchmarks

syn_model = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=0.1)
syn_model.fit(loader)
synth_df_e01 = syn_model.generate(count=75000).dataframe()

get_metrics("DP_e_01", synth_df_e01, clean_df)
score_e_01 = Benchmarks.evaluate(
    [("dpgan_eps_0.1", "dpgan", {"epsilon": 0.1, "n_iter": 10, "n_iter_min": 1})],
    loader,
    synthetic_size=75000,
    repeats=1,
    metrics={
        'stats': ['wasserstein_dist', 'prdc'],
        'performance': ['linear_model', 'mlp'],
        'detection': ['detection_mlp','detection_linear'],
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    }
    # synthetic_reuse_if_exists=False,
    # augmented_reuse_if_exists=False
)
Benchmarks.print(score_e_01)


[2023-12-11T18:33:39.477175+0000][4138][INFO] Encoding race 3843332182083652118
[2023-12-11T18:33:39.477175+0000][4138][INFO] Encoding race 3843332182083652118
[2023-12-11T18:33:39.490885+0000][4138][INFO] Encoding age 42226137574194105
[2023-12-11T18:33:39.490885+0000][4138][INFO] Encoding age 42226137574194105
[2023-12-11T18:33:42.085869+0000][4138][INFO] Encoding sex 7310119158062436415
[2023-12-11T18:33:42.085869+0000][4138][INFO] Encoding sex 7310119158062436415
[2023-12-11T18:33:42.105384+0000][4138][INFO] Encoding native-country 6190545641124984424
[2023-12-11T18:33:42.105384+0000][4138][INFO] Encoding native-country 6190545641124984424
[2023-12-11T18:33:44.483239+0000][4138][INFO] Encoding marital-status 1207444760813777779
[2023-12-11T18:33:44.483239+0000][4138][INFO] Encoding marital-status 1207444760813777779
[2023-12-11T18:33:44.502466+0000][4138][INFO] Encoding education 7923692313984985242
[2023-12-11T18:33:44.502466+0000][4138][INFO] Encoding education 792369231398498524

Statistics for dataset for mode: DP_e_01
Precision: 0.9923664122137404
Recall: 0.007651559741024131
AUROC: 0.5035593973452056
FTU: 0.0004310059014654201
DP: 0.0010705003660245


[2023-12-11T18:44:26.458093+0000][4138][CRITICAL] module disabled: /usr/local/lib/python3.10/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2023-12-11T18:44:26.458093+0000][4138][CRITICAL] module disabled: /usr/local/lib/python3.10/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2023-12-11T18:44:26.458093+0000][4138][CRITICAL] module disabled: /usr/local/lib/python3.10/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2023-12-11T18:44:27.334993+0000][4138][INFO] Encoding race 8839838288685913191
[2023-12-11T18:44:27.334993+0000][4138][INFO] Encoding race 8839838288685913191
[2023-12-11T18:44:27.346640+0000][4138][INFO] Encoding age 7983650266753259497
[2023-12-11T18:44:27.346640+0000][4138][INFO] Encoding age 7983650266753259497
[2023-12-11T18:44:29.315327+0000][4138][INFO] Encoding sex 4570725880838915145
[2023-12-11T18:44:29.315327+0000][4138][INFO] Encoding sex 4570725880838915145
[2023-12-11T18:44:29.327547+0000][4138][INFO] Encoding native-country 7154


Plugin : dpgan_eps_0.1


,min,max,mean,stddev,median,iqr,rounds,errors,durations
stats.prdc.precision,0.643461,0.643461,0.643461,0.0,0.643461,0.0,1,0,2.58
stats.prdc.recall,0.194099,0.194099,0.194099,0.0,0.194099,0.0,1,0,2.58
stats.prdc.density,0.198674,0.198674,0.198674,0.0,0.198674,0.0,1,0,2.58
stats.prdc.coverage,0.017736,0.017736,0.017736,0.0,0.017736,0.0,1,0,2.58
performance.linear_model.gt,0.808136,0.808136,0.808136,0.0,0.808136,0.0,1,0,1.02
performance.linear_model.syn_id,0.543231,0.543231,0.543231,0.0,0.543231,0.0,1,0,1.02
performance.linear_model.syn_ood,0.561444,0.561444,0.561444,0.0,0.561444,0.0,1,0,1.02
performance.mlp.gt,0.499586,0.499586,0.499586,0.0,0.499586,0.0,1,0,59.70
performance.mlp.syn_id,0.663829,0.663829,0.663829,0.0,0.663829,0.0,1,0,59.70
performance.mlp.syn_ood,0.669414,0.669414,0.669414,0.0,0.669414,0.0,1,0,59.70


In [26]:
# Evaluate dpgan for e=1
syn_dpgan_e1 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=1)
syn_dpgan_e1.fit(loader)
synth_df_e1 = syn_dpgan_e1.generate(count=75000).dataframe()
get_metrics("DP_e_1", synth_df_e1, clean_df)
score_e1 = Benchmarks.evaluate(
    [("dpgan_eps_1", "dpgan", {"epsilon": 1, "n_iter": 10, "n_iter_min": 1})],
    loader,
    synthetic_size=75000,
    repeats=1,
    metrics={
        'stats': ['wasserstein_dist', 'prdc'],
        'performance': ['linear_model', 'mlp'],
        'detection': ['detection_mlp','detection_linear'],
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    }
)
Benchmarks.print(score_e1)

[2023-12-11T18:55:42.972286+0000][4138][INFO] Encoding race 3843332182083652118
[2023-12-11T18:55:42.972286+0000][4138][INFO] Encoding race 3843332182083652118
[2023-12-11T18:55:42.985017+0000][4138][INFO] Encoding age 42226137574194105
[2023-12-11T18:55:42.985017+0000][4138][INFO] Encoding age 42226137574194105
[2023-12-11T18:55:45.443429+0000][4138][INFO] Encoding sex 7310119158062436415
[2023-12-11T18:55:45.443429+0000][4138][INFO] Encoding sex 7310119158062436415
[2023-12-11T18:55:45.461890+0000][4138][INFO] Encoding native-country 6190545641124984424
[2023-12-11T18:55:45.461890+0000][4138][INFO] Encoding native-country 6190545641124984424
[2023-12-11T18:55:48.078084+0000][4138][INFO] Encoding marital-status 1207444760813777779
[2023-12-11T18:55:48.078084+0000][4138][INFO] Encoding marital-status 1207444760813777779
[2023-12-11T18:55:48.100495+0000][4138][INFO] Encoding education 7923692313984985242
[2023-12-11T18:55:48.100495+0000][4138][INFO] Encoding education 792369231398498524

Statistics for dataset for mode: DP_e_1
Precision: 0.9310344827586207
Recall: 0.0015891701000588581
AUROC: 0.5002618199994167
FTU: 0.0009946290033817386
DP: -0.001817889316614081


[2023-12-11T19:06:37.083186+0000][4138][INFO] Encoding sex 4570725880838915145
[2023-12-11T19:06:37.083186+0000][4138][INFO] Encoding sex 4570725880838915145
[2023-12-11T19:06:37.102031+0000][4138][INFO] Encoding native-country 7154188036210242214
[2023-12-11T19:06:37.102031+0000][4138][INFO] Encoding native-country 7154188036210242214
[2023-12-11T19:06:39.132196+0000][4138][INFO] Encoding marital-status 9060001245496613666
[2023-12-11T19:06:39.132196+0000][4138][INFO] Encoding marital-status 9060001245496613666
[2023-12-11T19:06:39.154537+0000][4138][INFO] Encoding education 4958767644402141762
[2023-12-11T19:06:39.154537+0000][4138][INFO] Encoding education 4958767644402141762
[2023-12-11T19:06:40.059455+0000][4138][INFO] Encoding occupation 7487045939440258640
[2023-12-11T19:06:40.059455+0000][4138][INFO] Encoding occupation 7487045939440258640
[2023-12-11T19:06:42.373992+0000][4138][INFO] Encoding hours-per-week 54294204929279679
[2023-12-11T19:06:42.373992+0000][4138][INFO] Encodi


Plugin : dpgan_eps_1


,min,max,mean,stddev,median,iqr,rounds,errors,durations
stats.prdc.precision,0.710592,0.710592,0.710592,0.0,0.710592,0.0,1,0,2.39
stats.prdc.recall,0.515498,0.515498,0.515498,0.0,0.515498,0.0,1,0,2.39
stats.prdc.density,0.315266,0.315266,0.315266,0.0,0.315266,0.0,1,0,2.39
stats.prdc.coverage,0.049892,0.049892,0.049892,0.0,0.049892,0.0,1,0,2.39
performance.linear_model.gt,0.808136,0.808136,0.808136,0.0,0.808136,0.0,1,0,0.86
performance.linear_model.syn_id,0.571474,0.571474,0.571474,0.0,0.571474,0.0,1,0,0.86
performance.linear_model.syn_ood,0.606953,0.606953,0.606953,0.0,0.606953,0.0,1,0,0.86
performance.mlp.gt,0.499586,0.499586,0.499586,0.0,0.499586,0.0,1,0,57.23
performance.mlp.syn_id,0.664039,0.664039,0.664039,0.0,0.664039,0.0,1,0,57.23
performance.mlp.syn_ood,0.674491,0.674491,0.674491,0.0,0.674491,0.0,1,0,57.23


In [27]:
# Evaluate dpgan for e=10
syn_dpgan_e10 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=10)
syn_dpgan_e10.fit(loader)
synth_df_e10 = syn_dpgan_e10.generate(count=75000).dataframe()
get_metrics("DP_e_10", synth_df_e10, clean_df)
score_e10 = Benchmarks.evaluate(
    [("dpgan_eps_10", "dpgan", {"epsilon": 10, "n_iter": 10, "n_iter_min": 1})],
    loader,
    synthetic_size=75000,
    repeats=1,
    metrics={
        'stats': ['wasserstein_dist', 'prdc'],
        'performance': ['linear_model', 'mlp'],
        'detection': ['detection_mlp','detection_linear'],
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    }
)
Benchmarks.print(score_e10)

[2023-12-11T19:17:11.861932+0000][4138][INFO] Encoding race 3843332182083652118
[2023-12-11T19:17:11.861932+0000][4138][INFO] Encoding race 3843332182083652118
[2023-12-11T19:17:11.871810+0000][4138][INFO] Encoding age 42226137574194105
[2023-12-11T19:17:11.871810+0000][4138][INFO] Encoding age 42226137574194105
[2023-12-11T19:17:14.144774+0000][4138][INFO] Encoding sex 7310119158062436415
[2023-12-11T19:17:14.144774+0000][4138][INFO] Encoding sex 7310119158062436415
[2023-12-11T19:17:14.165807+0000][4138][INFO] Encoding native-country 6190545641124984424
[2023-12-11T19:17:14.165807+0000][4138][INFO] Encoding native-country 6190545641124984424
[2023-12-11T19:17:16.413590+0000][4138][INFO] Encoding marital-status 1207444760813777779
[2023-12-11T19:17:16.413590+0000][4138][INFO] Encoding marital-status 1207444760813777779
[2023-12-11T19:17:16.427373+0000][4138][INFO] Encoding education 7923692313984985242
[2023-12-11T19:17:16.427373+0000][4138][INFO] Encoding education 792369231398498524

Statistics for dataset for mode: DP_e_10
Precision: 1.0
Recall: 0.004355503237198352
AUROC: 0.5021777516185991
FTU: 0.004707910616006896
DP: -0.004649256710887426


[2023-12-11T19:27:48.028348+0000][4138][INFO] Encoding sex 4570725880838915145
[2023-12-11T19:27:48.028348+0000][4138][INFO] Encoding sex 4570725880838915145
[2023-12-11T19:27:48.040830+0000][4138][INFO] Encoding native-country 7154188036210242214
[2023-12-11T19:27:48.040830+0000][4138][INFO] Encoding native-country 7154188036210242214
[2023-12-11T19:27:49.872681+0000][4138][INFO] Encoding marital-status 9060001245496613666
[2023-12-11T19:27:49.872681+0000][4138][INFO] Encoding marital-status 9060001245496613666
[2023-12-11T19:27:49.885295+0000][4138][INFO] Encoding education 4958767644402141762
[2023-12-11T19:27:49.885295+0000][4138][INFO] Encoding education 4958767644402141762
[2023-12-11T19:27:50.542029+0000][4138][INFO] Encoding occupation 7487045939440258640
[2023-12-11T19:27:50.542029+0000][4138][INFO] Encoding occupation 7487045939440258640
[2023-12-11T19:27:52.404680+0000][4138][INFO] Encoding hours-per-week 54294204929279679
[2023-12-11T19:27:52.404680+0000][4138][INFO] Encodi


Plugin : dpgan_eps_10


,min,max,mean,stddev,median,iqr,rounds,errors,durations
stats.prdc.precision,0.706448,0.706448,0.706448,0.0,0.706448,0.0,1,0,2.45
stats.prdc.recall,0.644787,0.644787,0.644787,0.0,0.644787,0.0,1,0,2.45
stats.prdc.density,0.249362,0.249362,0.249362,0.0,0.249362,0.0,1,0,2.45
stats.prdc.coverage,0.019891,0.019891,0.019891,0.0,0.019891,0.0,1,0,2.45
performance.linear_model.gt,0.808136,0.808136,0.808136,0.0,0.808136,0.0,1,0,0.75
performance.linear_model.syn_id,0.577607,0.577607,0.577607,0.0,0.577607,0.0,1,0,0.75
performance.linear_model.syn_ood,0.547005,0.547005,0.547005,0.0,0.547005,0.0,1,0,0.75
performance.mlp.gt,0.499586,0.499586,0.499586,0.0,0.499586,0.0,1,0,53.74
performance.mlp.syn_id,0.642139,0.642139,0.642139,0.0,0.642139,0.0,1,0,53.74
performance.mlp.syn_ood,0.651588,0.651588,0.651588,0.0,0.651588,0.0,1,0,53.74


In [28]:
#save csv

synth_df_e01.to_csv('dpgan_e01.csv', index=False)
synth_df_e1.to_csv('dpgan_e1.csv', index=False)
synth_df_e10.to_csv('dpgan_e10.csv', index=False)